<a href="https://colab.research.google.com/github/hyona-yu/HAR_python/blob/master/har_checking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import numpy as np
import pandas as pd
import random

In [ ]:
#! git clone https://github.com/hyona-yu/Dataset.git


In [ ]:
url = 'Dataset/UCI HAR Dataset/'

In [ ]:
feature_name_df = pd.read_csv(url + 'features.txt', sep='\s+', header=None, 
                              names=['column_index','column_name'])

feature_name = feature_name_df.iloc[:,1].values.tolist()
train_csv = pd.read_csv(url + 'train/X_train.txt', sep ='\s+', header = None)
train_csv.columns = feature_name
test_csv = pd.read_csv(url + 'test/X_test.txt', sep = '\s+', header= None)
test_csv.columns = feature_name
train_y = pd.read_csv(url + 'train/y_train.txt', header = None, names= ['label'])
test_y =pd.read_csv(url + 'test/y_test.txt', header =None, names= ['label'])

In [ ]:
train_csv

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,tBodyAcc-max()-Y,tBodyAcc-max()-Z,tBodyAcc-min()-X,tBodyAcc-min()-Y,tBodyAcc-min()-Z,tBodyAcc-sma(),tBodyAcc-energy()-X,tBodyAcc-energy()-Y,tBodyAcc-energy()-Z,tBodyAcc-iqr()-X,tBodyAcc-iqr()-Y,tBodyAcc-iqr()-Z,tBodyAcc-entropy()-X,tBodyAcc-entropy()-Y,tBodyAcc-entropy()-Z,"tBodyAcc-arCoeff()-X,1","tBodyAcc-arCoeff()-X,2","tBodyAcc-arCoeff()-X,3","tBodyAcc-arCoeff()-X,4","tBodyAcc-arCoeff()-Y,1","tBodyAcc-arCoeff()-Y,2","tBodyAcc-arCoeff()-Y,3","tBodyAcc-arCoeff()-Y,4","tBodyAcc-arCoeff()-Z,1","tBodyAcc-arCoeff()-Z,2","tBodyAcc-arCoeff()-Z,3","tBodyAcc-arCoeff()-Z,4","tBodyAcc-correlation()-X,Y","tBodyAcc-correlation()-X,Z","tBodyAcc-correlation()-Y,Z",...,fBodyBodyAccJerkMag-energy(),fBodyBodyAccJerkMag-iqr(),fBodyBodyAccJerkMag-entropy(),fBodyBodyAccJerkMag-maxInds,fBodyBodyAccJerkMag-meanFreq(),fBodyBodyAccJerkMag-skewness(),fBodyBodyAccJerkMag-kurtosis(),fBodyBodyGyroMag-mean(),fBodyBodyGyroMag-std(),fBodyBodyGyroMag-mad(),fBodyBodyGyroMag-max(),fBodyBodyGyroMag-min(),fBodyBodyGyroMag-sma(),fBodyBodyGyroMag-energy(),fBodyBodyGyroMag-iqr(),fBodyBodyGyroMag-entropy(),fBodyBodyGyroMag-maxInds,fBodyBodyGyroMag-meanFreq(),fBodyBodyGyroMag-skewness(),fBodyBodyGyroMag-kurtosis(),fBodyBodyGyroJerkMag-mean(),fBodyBodyGyroJerkMag-std(),fBodyBodyGyroJerkMag-mad(),fBodyBodyGyroJerkMag-max(),fBodyBodyGyroJerkMag-min(),fBodyBodyGyroJerkMag-sma(),fBodyBodyGyroJerkMag-energy(),fBodyBodyGyroJerkMag-iqr(),fBodyBodyGyroJerkMag-entropy(),fBodyBodyGyroJerkMag-maxInds,fBodyBodyGyroJerkMag-meanFreq(),fBodyBodyGyroJerkMag-skewness(),fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)"
0,0.288585,-0.020294,-0.132905,-0.995279,-0.983111,-0.913526,-0.995112,-0.983185,-0.923527,-0.934724,-0.567378,-0.744413,0.852947,0.685845,0.814263,-0.965523,-0.999945,-0.999863,-0.994612,-0.994231,-0.987614,-0.943220,-0.407747,-0.679338,-0.602122,0.929294,-0.853011,0.359910,-0.058526,0.256892,-0.224848,0.264106,-0.095246,0.278851,-0.465085,0.491936,-0.190884,0.376314,0.435129,0.660790,...,-0.999918,-0.991364,-1.000000,-0.936508,0.346989,-0.516080,-0.802760,-0.980135,-0.961309,-0.973653,-0.952264,-0.989498,-0.980135,-0.999240,-0.992656,-0.701291,-1.000000,-0.128989,0.586156,0.374605,-0.991990,-0.990697,-0.989941,-0.992448,-0.991048,-0.991990,-0.999937,-0.990458,-0.871306,-1.000000,-0.074323,-0.298676,-0.710304,-0.112754,0.030400,-0.464761,-0.018446,-0.841247,0.179941,-0.058627
1,0.278419,-0.016411,-0.123520,-0.998245,-0.975300,-0.960322,-0.998807,-0.974914,-0.957686,-0.943068,-0.557851,-0.818409,0.849308,0.685845,0.822637,-0.981930,-0.999991,-0.999788,-0.998405,-0.999150,-0.977866,-0.948225,-0.714892,-0.500930,-0.570979,0.611627,-0.329549,0.284213,0.284595,0.115705,-0.090963,0.294310,-0.281211,0.085988,-0.022153,-0.016657,-0.220643,-0.013429,-0.072692,0.579382,...,-0.999867,-0.991134,-1.000000,-0.841270,0.532061,-0.624871,-0.900160,-0.988296,-0.983322,-0.982659,-0.986321,-0.991829,-0.988296,-0.999811,-0.993979,-0.720683,-0.948718,-0.271958,-0.336310,-0.720015,-0.995854,-0.996399,-0.995442,-0.996866,-0.994440,-0.995854,-0.999981,-0.994544,-1.000000,-1.000000,0.158075,-0.595051,-0.861499,0.053477,-0.007435,-0.732626,0.703511,-0.844788,0.180289,-0.054317
2,0.279653,-0.019467,-0.113462,-0.995380,-0.967187,-0.978944,-0.996520,-0.963668,-0.977469,-0.938692,-0.557851,-0.818409,0.843609,0.682401,0.839344,-0.983478,-0.999969,-0.999660,-0.999470,-0.997130,-0.964810,-0.974675,-0.592235,-0.485821,-0.570979,0.273025,-0.086309,0.337202,-0.164739,0.017150,-0.074507,0.342256,-0.332564,0.239281,-0.136204,0.173863,-0.299493,-0.124698,-0.181105,0.608900,...,-0.999845,-0.986658,-1.000000,-0.904762,0.660795,-0.724697,-0.928539,-

In [ ]:
print(len(train_csv), len(train_y))


7352 7352


In [ ]:
test_y


,label
0,5
1,5
2,5
3,5
4,5
...,...
2942,2
2943,2
2944,2
2945,2


In [ ]:
feature_name_df.head()

,column_index,column_name
0,1,tBodyAcc-mean()-X
1,2,tBodyAcc-mean()-Y
2,3,tBodyAcc-mean()-Z
3,4,tBodyAcc-std()-X
4,5,tBodyAcc-std()-Y


In [ ]:
train_y.describe()
test_y.describe()

,label
count,2947.000000
mean,3.577876
std,1.740348
min,1.000000
25%,2.000000
50%,4.000000
75%,5.000000
max,6.000000


#preprocessing version

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
new_train_csv = scaler.fit_transform(np.array(train_csv), np.array(train_y))
new_test_csv = scaler.transform(np.array(test_csv))

#SVM

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

clf = SVC(random_state = 42)
params = {
    'C' : [0.5,1,10]
}
cv = GridSearchCV(clf, param_grid = params)
cv.fit(train_csv, train_y)


/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

GridSearchCV(cv=None, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=42, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=None, param_grid={'C': [0.5, 1, 10]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [ ]:
pred = cv.predict(test_csv)

acc = accuracy_score(pred, test_y)
print("SVM :",acc)

SVM : 0.9619952494061758


#CNN

In [ ]:
device ='cuda' if torch.cuda.is_available() else 'cpu'
random.seed(42)
torch.manual_seed(42)
if device =='cuda':
    torch.cuda.manual_seed_all(42)


class CNN(torch.nn.Module):
    def __init__(self, input_size, output_size):
        super(CNN, self).__init__()
# filter 수 늘리는게 중요? kernel size 크게
        self.layer1 = torch.nn.Sequential(torch.nn.Conv1d(input_size ,128, kernel_size= 1, stride= 1), torch.nn.MaxPool1d(kernel_size= 1), 
                                          torch.nn.ReLU()) #
        self.layer2 = torch.nn.Sequential(torch.nn.Conv1d(128,128, kernel_size= 1, stride= 1, padding =0),
                                          torch.nn.MaxPool1d(kernel_size= 1, stride = 1, padding = 0), 
                                          torch.nn.ReLU())

        self.layer3 = torch.nn.Linear(128 , output_size, bias = True) #(input_size - kernel_size + 2*padding_size)/stride + 1

        torch.nn.init.kaiming_uniform(self.layer3.weight)

    def forward(self, x):
        out = self.layer1(x)
        #print(out.shape)
        out = self.layer2(out)
        #print(out.shape)
        out = out.view(out.size(0), -1)
        #print(out.shape)
        out = self.layer3(out)
        return out


In [ ]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder()
encoder.fit(train_y)


OneHotEncoder(categories='auto', drop=None, dtype=<class 'numpy.float64'>,
              handle_unknown='error', sparse=True)

In [ ]:
train_L = encoder.transform(train_y).toarray()

In [ ]:
model = CNN(train_csv.shape[1], train_L.shape[1]).to(device)
criterion = torch.nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 1e-3)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: UserWarning: nn.init.kaiming_uniform is now deprecated in favor of nn.init.kaiming_uniform_.


In [ ]:
model

CNN(
  (layer1): Sequential(
    (0): Conv1d(561, 128, kernel_size=(1,), stride=(1,))
    (1): MaxPool1d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
    (2): ReLU()
  )
  (layer2): Sequential(
    (0): Conv1d(128, 128, kernel_size=(1,), stride=(1,))
    (1): MaxPool1d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
    (2): ReLU()
  )
  (layer3): Linear(in_features=128, out_features=6, bias=True)
)

In [ ]:
train_csv
# train_y.shape
# train_y
train_csv.shape[1]
train_L.shape

(7352, 6)

In [ ]:
train_L

array([[0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1., 0.],
       ...,
       [0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.]])

In [ ]:
np.array(train_csv).dtype#describe()

dtype('float64')

In [ ]:
#train_D = np.expand_dims(np.array(train_csv), axis = 2)

In [ ]:
train_D = torch.FloatTensor(np.array(train_csv))
train_L = torch.FloatTensor(np.array(train_L))

In [ ]:
import torch.utils
train_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(train_D, train_L), batch_size = 100, shuffle= True, drop_last = True)

In [25]:
running_loss =0
for e in range(1001):
  for i , data in enumerate(train_loader):
    x, y = data
    #print(x.shape)
    x = x.resize(100,561,1).to(device)
    y = y.to(device)
    optimizer.zero_grad()
    h = model(x)
    loss = criterion(h, y)
    loss.backward()
    optimizer.step()
    running_loss += loss.item()
  if (e+1) % 100 == 0:
    print('[%d] loss : %.3f' %(e, running_loss / 100))
    running_loss =0

print('fin')

/usr/local/lib/python3.6/dist-packages/torch/tensor.py:358: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


[99] loss : 5.622
[199] loss : 3.581
[299] loss : 2.814
[399] loss : 2.395
[499] loss : 2.112
[599] loss : 1.891
[699] loss : 1.717
[799] loss : 1.576
[899] loss : 1.461
[999] loss : 1.364
fin


In [26]:
from torch.utils.tensorboard import SummaryWriter
import torchvision
writer = SummaryWriter('runs/har_cnn')
dataiter = iter(train_loader)
img, labels = dataiter.next()
img_grid = torchvision.utils.make_grid(img)

#matplotlib_imshow(img_grid, one_channel = True)
writer.add_image('har_cnn', img_grid)

In [27]:
# ! tensorboard --logdir=data/ --host localhost --port 8088
# ! tensorboard --logdir=runs

In [28]:
PATH = './file.pt'
torch.save({
            'epoch': 1001,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss,

            }, PATH)

In [29]:
with torch.no_grad():
  test_D = torch.FloatTensor(np.array(test_csv))
  model.eval()
  pred=  model(test_D.view(-1,561,1).to(device))



In [30]:
pred.shape

torch.Size([2947, 6])

In [31]:
pred[0]
np.array(test_y).reshape(2947,)[1]
#test_y = np.array(test_y).reshape(1, 2947)

5

In [32]:
test_y

,label
0,5
1,5
2,5
3,5
4,5
...,...
2942,2
2943,2
2944,2
2945,2


In [33]:
acc = [pred[i].argmax()+1 == np.array(test_y).reshape(2947,)[i] for i in range(len(test_D))]
acc = np.array(acc, dtype=np.float32)
real_acc = acc.mean()
print('CNN : ', real_acc)

CNN :  0.93926024


#LSTM

In [34]:
import random
device = 'cuda' if torch.cuda.is_available() else 'cpu'

random.seed(777)
torch.manual_seed(777)
if device == 'cuda':
  torch.cuda.manual_seed_all(777)

In [35]:
class LSTM(torch.nn.Module):
  def __init__(self, n_class, bi):
    super(LSTM, self).__init__()

    self.lstm = torch.nn.LSTM(561, 128, 2,dropout = 0.1, bidirectional = bi)
    self.l1 = torch.nn.Linear(128, n_class)
    self.dropout = torch.nn.Dropout(0.1)
    self.tanh = torch.nn.Tanh()
    self.softmax = torch.nn.Softmax(dim = 1)
    self.h = torch.randn(2,1,128).to(device)
    self.c = torch.randn(2,1,128).to(device)
  def forward(self, x):
    #print(x.shape)
    x = x.permute(1,0,2)# 축 바꾸기. 전치행렬처럼
    #print(x.shape)

    out, hidden = self.lstm(x,(self.h,self.c) )
    #print(out.shape)
    out = self.tanh(out)
    out = self.dropout(out)
    out = out.contiguous().view(-1, 128)
    #print(out.shape)
    out = self.l1(out)
    #out = self.softmax(out)
    return out


In [36]:
model = LSTM(train_L.shape[1], False).to(device)

In [37]:
model

LSTM(
  (lstm): LSTM(561, 128, num_layers=2, dropout=0.1)
  (l1): Linear(in_features=128, out_features=6, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
  (tanh): Tanh()
  (softmax): Softmax(dim=1)
)

In [38]:
criterion = torch.nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 1e-3)

In [ ]:
running_loss =0
for e in range(1001):
  for i , data in enumerate(train_loader):
    x, y = data
    x = x.resize(1,100,561).to(device)
    y = y.to(device)
    optimizer.zero_grad()
    h = model(x)
    loss = criterion(h, y)
    loss.backward()
    optimizer.step()
    running_loss += loss.item()
  if e % 100 == 0:
    print('[%d] loss : %.3f' %(e, running_loss / 100))
    running_loss =0

print('fin')

/usr/local/lib/python3.6/dist-packages/torch/tensor.py:358: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


[0] loss : 0.133
[100] loss : 10.184
[200] loss : 9.777
[300] loss : 9.488
[400] loss : 9.160
[500] loss : 8.834


In [ ]:
test_D.shape

In [ ]:
with torch.no_grad():
  test_D = torch.FloatTensor(np.array(test_csv))
  model.eval()
  pred_lstm = model(test_D.view(1,-1,561).to(device))



In [ ]:
print(pred_lstm)

In [ ]:
acc = [pred_lstm[i].argmax()+1 == np.array(test_y).reshape(2947,)[i] for i in range(len(test_D))]
acc = np.array(acc, dtype=np.float32)
real_acc = acc.mean()
print('LSTM : ', real_acc)

#CNN + LSTM

In [ ]:
class CNN_LSTM(torch.nn.Module):
    def __init__(self, input_size, output_size, n_class, bi):
        super(CNN_LSTM, self).__init__()
        self.layer1 = torch.nn.Sequential(torch.nn.Conv1d(input_size ,128, kernel_size= 1, stride= 1, padding =0), torch.nn.ReLU()) #
        self.layer2 = torch.nn.Sequential(torch.nn.Conv1d(128,256, kernel_size= 1, stride= 1, padding =0), torch.nn.ReLU())
        self.layer3 = torch.nn.LSTM(256,  64, 2,dropout = 0.3, bidirectional = bi)
        if bi:
          self.layer3 = torch.nn.LSTM(64, 32, 2, dropout = 0.3, bidirectional = bi)
        self.bi = bi
        #self.h = torch.randn(2,1,64).to(device)
        #self.c = torch.randn(2,1,64).to(device)
        self.layer4 = torch.nn.Linear(64, n_class)
        if bi:
          self.layer = torch.nn.Linear(32, n_class)
        torch.nn.init.xavier_uniform(self.layer4.weight)
        self.dropout = torch.nn.Dropout(0.3)

    def forward(self, x):
      out = self.layer1(x)
      #print('first layer,', out.shape)
      out = self.layer2(out)
      #print('sec layer', out.shape)
      out = out.permute(2, 0, 1)
      #print('premute', out.shape)
      out, hidden = self.layer3(out)#,(self.h,self.c))
      #print('third layer', out.shape)
      out = self.dropout(out)
      out = out.contiguous().view(-1, 64)
      #print('contig', out.view)
      out = self.layer4(out)
      #print('last layer', out)

      return out



#       first layer, torch.Size([2947, 64, 1])
# sec layer torch.Size([2947, 64, 1])
# premute torch.Size([1, 2947, 64])
# third layer torch.Size([1, 2947, 64]
      


In [ ]:
model = CNN_LSTM(train_D.shape[1], train_L.shape[1], train_L.shape[1] , False).to(device)
model

In [ ]:
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(params = model.parameters(), lr = 1e-3)

In [ ]:
running_loss = 0
for e in range(1001):
  for i, data in enumerate(train_loader):
    x, y= data
    x = x.view(-1,561,1).to(device)
    y = y.to(device)
    optimizer.zero_grad()
    h = model(x)
    loss = criterion(h, y)
    loss.backward()
    optimizer.step()
    running_loss += loss.item()
  if e % 100 ==0:
    print('[%d] loss : %.3f' %(e, running_loss/100))
    running_loss =0
print('fin')

In [ ]:
with torch.no_grad():
  test_D = torch.FloatTensor(np.array(test_csv))
  model.eval()
  pred_cnn_lstm = model(test_D.reshape(-1,561,1).to(device))

In [ ]:
acc = [pred_cnn_lstm[i].argmax()+1 == np.array(test_y).reshape(2947,)[i] for i in range(len(test_D))]
acc = np.array(acc, dtype=np.float32)
real_acc = acc.mean()
print('LSTM_CNN : ', real_acc)

In [ ]:
acc = [pred_cnn_lstm[i].argmax()+1 == np.array(test_y).reshape(2947,)[i] for i in range(len(test_D))]
acc = np.array(acc, dtype=np.float32)
real_acc = acc.mean()
print('LSTM_CNN : ', real_acc)

#bi-LSTM_CNN

In [ ]:
model = CNN_LSTM(train_D.shape[1], train_L.shape[1], train_L.shape[1] , True).to(device)
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(params = model.parameters(), lr = 1e-2)

In [ ]:
running_loss = 0
for e in range(1001):
  for i, data in enumerate(train_loader):
    x, y= data
    #print(x.shape)
    x = x.view(-1,561,1).to(device)
    y = y.to(device)
    optimizer.zero_grad()
    h = model(x)
    loss = criterion(h, y)
    loss.backward()
    optimizer.step()
    running_loss += loss.item()
  if e % 100 ==0:
    print('[%d] loss : %.3f' %(e, running_loss/100))
    running_loss =0
print('fin')

In [ ]:
with torch.no_grad():
  test_D = torch.FloatTensor(np.array(test_csv))
  model.eval()
  pred_cnn_lstm = model(test_D.reshape(-1,561,1).to(device))

In [ ]:
acc = [pred_cnn_lstm[i].argmax()+1 == np.array(test_y).reshape(2947,)[i] for i in range(len(test_D))]
acc = np.array(acc, dtype=np.float32)
real_acc = acc.mean()
print('bi-LSTM_CNN : ', real_acc)

#LSTM + CNN

In [ ]:
class LSTM_CNN(torch.nn.Module):
  def __init__(self):
    super(LSTM_CNN, self).__init__()
    self.lstm1 = torch.nn.LSTM(561, 32, 2)
    #self.lstm2 = torch.nn.LSTM(32, 32)
    self.conv1 = torch.nn.Sequential(torch.nn.Conv1d(32, 64,kernel_size = 5,stride = 2) ,torch.nn.MaxPool1d(2))
    self.conv2 = torch.nn.Sequential( torch.nn.Conv1d(64, 128, 3, 1),
                                     torch.nn.AvgPool1d(1))
    self.bn = torch.nn.BatchNorm2d(128)
    self.l1 = torch.nn.Linear(128, 6)

  def forward(self, x):
    out, _ = self.lstm1(x)
    print('lstm1')
    #out = self.lstm2(out)
    #print('lstm2')
    out = out.permute(0, 2, 1)

    out = self.conv1(out)
    print('conv1')
    out = self.conv2(out)
    print('conv2')
    out = self.bn(out)
    print('batchnormal')
    out= self.l1(out)
    print('linear')

    return out

In [ ]:
model = LSTM_CNN().to(device)

In [ ]:
criterion = torch.nn.MSELoss()
optimizer = torch.optim.SGD(params = model.parameters(), lr = 1e-3)

In [ ]:
running_loss = 0
for e in range(1001):
  for i, data in enumerate(train_loader):
    x, y= data
    #print(x.shape)
    x = x.view(-1,1,561).to(device)
    y = y.to(device)
    optimizer.zero_grad()
    h = model(x)
    loss = criterion(h, y)
    loss.backward()
    optimizer.step()
    running_loss += loss.item()
  if e % 100 ==0:
    print('[%d] loss : %.3f' %(e, running_loss/100))
    running_loss =0
print('fin')